<a href="https://colab.research.google.com/github/PriyankaSharma0925/NLP/blob/main/Introduction_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow
tensorflow.__version__

'2.8.0'

In [1]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.8.0


In [3]:
pip install tensorflow==2.7

     |████████████████████████████████| 489.6 MB 24 kB/s 
     |████████████████████████████████| 463 kB 60.3 MB/s 
     |████████████████████████████████| 1.3 MB 68.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


In [4]:

# NLP problem also known as seq2seq

#Check the GPU

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-14f6ef5d-0057-ad21-c08b-49f7d91fd92f)


In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-05-10 06:41:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-10 06:41:15 (113 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

In [7]:
#importing the dataset: Kaggle's Introduction to NLP 

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip


--2022-05-10 06:41:21--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.144, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-05-10 06:41:21 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [8]:
unzip_data('nlp_getting_started.zip')

Visualizing the text data.

In [9]:
import pandas as pd

In [10]:
train_df=pd.read_csv("train.csv")

In [11]:
test_df=pd.read_csv("test.csv")

In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
shuffled_train_dataset=train_df.sample(frac=1,random_state=42)
shuffled_train_dataset.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
X_train=test_df['text']
X_train
Y_train=shuffled_train_dataset["target"]

In [16]:
import tensorflow as tf

In [17]:
from tensorflow.keras.layers import TextVectorization

In [18]:
textvectorizer=TextVectorization(output_sequence_length=15,pad_to_max_tokens=True,max_tokens=1000)

In [19]:
sent=test_df['text'][0]

In [22]:
textvectorizer.adapt(X_train)

In [23]:
textvectorizer("Its raining heavily outside")

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([ 48,   1,   1, 999,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

In [25]:
import random

In [28]:
rand_sent=random.choice(X_train)
rand_sent

"It's a trap not a fucking game. U ain't trapping if u ain't trapped"

In [29]:
textvectorizer(rand_sent)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([ 48,   3,   1,  31,   3, 387, 247, 174, 867,   1,  42, 174, 867,
       256,   0])>

**Get the total unique words.**

In [30]:
vocab=textvectorizer.get_vocabulary()
print(vocab)

['', '[UNK]', 'the', 'a', 'to', 'in', 'of', 'and', 'i', 'is', 'on', 'for', 'you', 'my', 'it', 'by', 'with', 'this', 'that', 'at', 'are', 'from', 'be', 'was', 'as', 'like', 'your', 'out', 'amp', 'no', 'just', 'not', 'up', 'have', 'we', 'im', 'will', 'me', 'so', 'but', 'all', 'after', 'if', 'fire', 'more', 'get', 'when', 'via', 'its', 'an', 'new', 'how', 'about', 'he', 'has', 'news', 'or', 'one', 'now', 'people', 'they', 'what', 'who', 'would', 'do', '2', 'us', 'can', 'dont', 'emergency', 'over', 'were', 'there', 'into', 'attack', 'her', 'why', 'his', 'video', 'disaster', 'first', 'suicide', 'been', 'rt', 'police', 'off', 'fires', 'our', 'had', 'watch', 'them', 'still', 'some', '3', 'time', 'than', 'storm', 'hiroshima', 'full', 'got', 'crash', 'two', 'burning', 'going', 'today', 'see', 'love', 'body', 'last', 'know', 'go', 'think', 'make', 'life', 'world', 'nuclear', 'cant', 'buildings', 'need', 'forest', 'dead', 'bomb', 'back', 'right', 'even', 'bombing', '1', 'youtube', 'only', 'him', 

In [31]:
type(vocab)

list

In [32]:
vocab[-5:]

['picture', 'photos', 'peace', 'pakistan', 'outside']

In [35]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=len(vocab),output_dim=128,input_length=15)
embedding

In [39]:
rand_sent=random.choice(X_train)
embedding(textvectorizer(rand_sent))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.0122233 ,  0.04490808, -0.00791281, ..., -0.0149537 ,
         0.01919958, -0.03136011],
       [ 0.01238673, -0.04723901, -0.00113233, ...,  0.02010835,
         0.02661991,  0.03108094],
       [-0.03603124,  0.02543193, -0.01219287, ...,  0.029452  ,
         0.04727442, -0.03393813],
       ...,
       [ 0.01238673, -0.04723901, -0.00113233, ...,  0.02010835,
         0.02661991,  0.03108094],
       [ 0.01238673, -0.04723901, -0.00113233, ...,  0.02010835,
         0.02661991,  0.03108094],
       [ 0.00387968,  0.00064818,  0.01900084, ...,  0.02280777,
         0.00505747,  0.02089237]], dtype=float32)>